In [1]:
import tensorflow as tf
import os

from keras_invariance_checker import InvarianceCheckerKeras
from pytorch_invariance_checker import InvarianceCheckerPytorch

In [10]:
import sys

sys.path.append(os.path.abspath('../'))
from utils import Metrics, plot_confusion_matrix
from training_scripts.vit_model import ViTForImageClassification
from training_scripts.vit_helpers import *

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

keras_checker = InvarianceCheckerKeras()
pytorch_checker = InvarianceCheckerPytorch()

<br/>
<br/>

## Eurosat Data

### ResNet

In [3]:
model_dir = "../eurosat/ResNet"
eurosat_ResNet = tf.keras.models.load_model(
    os.path.join(model_dir, "17-0.94.keras")
)

2024-06-07 13:05:48.341598: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-06-07 13:05:48.341619: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-06-07 13:05:48.341642: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-06-07 13:05:48.341711: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-07 13:05:48.341740: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
rotated_loader, normal_loader = keras_checker.transform_test_set(path = "../eurosat/data/test", image_size=(224,224,3), batch_size=128)

Found 2700 images belonging to 10 classes.
Found 2700 images belonging to 10 classes.


In [5]:
rotated_predictions, normal_predictions = keras_checker.predict_models(model = eurosat_ResNet, rotated_testset=rotated_loader, normal_testset=normal_loader)

Rotated Testset:
--------------------------------------------------


2024-06-07 13:05:55.858329: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


22/22 [==============================] - 21s 727ms/step
Accuracy: 0.9274
--------------------------------------------------
--------------------------------------------------
Normal Testset:
--------------------------------------------------
22/22 [==============================] - 11s 497ms/step
Accuracy: 0.9437


In [8]:
result= keras_checker.calculate_invariance(rotated_predictions, normal_predictions)
print(f"Invariance: {result:.4f}")

Invariance: 0.9356


### ViT

In [15]:
vit_model = load_model(path = "../eurosat/ViT/37_val_acc0.7911.pth", model = ViTForImageClassification(10))

In [37]:
_, _, test_rotated_loader, test_normal_loader = pytorch_checker.transform_test_set(path = "../eurosat/data/test")
rotated_predictions, normal_predictions = pytorch_checker.predict_models(vit_model, test_rotated_loader=test_rotated_loader, test_normal_loader=test_normal_loader, device = device)

Rotated Testset:
--------------------------------------------------


100%|██████████| 43/43 [02:39<00:00,  3.70s/it]


Validation Loss: 0.6224 | Accuracy: 0.7748
--------------------------------------------------
--------------------------------------------------
Normal Testset:
--------------------------------------------------


100%|██████████| 43/43 [02:39<00:00,  3.70s/it]

Validation Loss: 0.5922 | Accuracy: 0.7926


In [66]:
class InvarianceCheckerPytorch:
    def __init__(self) -> None:
        self.rotated_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomChoice([
                transforms.RandomRotation((90, 90)),
                transforms.RandomRotation((180, 180)),
                transforms.RandomRotation((270, 270))
            ]),
            transforms.ToTensor(),
        ])
        
        self.normal_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])
    
    def transform_test_set(self, path: str, batch_size: int = 64):
        test_rotated_ds = datasets.ImageFolder(path, transform=self.rotated_transform)
        test_normal_ds = datasets.ImageFolder(path, transform=self.normal_transform)
        
        test_rotated_loader = data.DataLoader(test_rotated_ds, batch_size=batch_size, shuffle=False, num_workers=4)
        test_normal_loader = data.DataLoader(test_normal_ds, batch_size=batch_size, shuffle=False, num_workers=4)
        
        return test_rotated_ds, test_normal_ds, test_rotated_loader, test_normal_loader
    
    def predict_models(self, model, test_rotated_loader, test_normal_loader, device):
        print(f"Rotated Testset:")
        print("-"* 50)
        rotated_predictions = predict_testing(test_rotated_loader, model = model, device = device)
        print("-"* 50)
        print("-"* 50)
        print(f"Normal Testset:")
        print("-"* 50)
        normal_predictions = predict_testing(test_normal_loader, model = model, device = device)
        
        return rotated_predictions, normal_predictions
    
    def calculate_invariance(self, rotated_predictions, normal_predictions):
        count = 0 
        
        for i, v in zip(rotated_predictions[0], normal_predictions[0]):
            if i == v:
                count += 1
        return (count)/len(rotated_predictions) 

In [77]:
result = pytorch_checker.calculate_invariance(rotated_predictions, normal_predictions)
print(f"Invariance: {result:.4f}")

Invariance: 0.8789


<br/>
<br/>

## Places Dataset

### ResNet

In [ ]:
model_dir = "../places/ResNet/augment"
places_ResNet_augment = tf.keras.models.load_model(
    os.path.join(model_dir, "08-0.84.keras")
)

model_dir = "../places/ResNet/normal"
places_ResNet_normal = tf.keras.models.load_model(
    os.path.join(model_dir, "22-0.86.keras")
)

### Resnet - Agumented

In [79]:
rotated_loader, normal_loader = keras_checker.transform_test_set(path = "../places/data/test", image_size=(224,224,3), batch_size=128)
rotated_predictions, normal_predictions = keras_checker.predict_models(model = places_ResNet_augment, rotated_testset=rotated_loader, normal_testset=normal_loader)

Found 6250 images belonging to 10 classes.
Found 6250 images belonging to 10 classes.
Rotated Testset:
--------------------------------------------------
49/49 [==============================] - 36s 621ms/step
Accuracy: 0.8398
--------------------------------------------------
--------------------------------------------------
Normal Testset:
--------------------------------------------------
49/49 [==============================] - 26s 524ms/step
Accuracy: 0.8446


In [80]:
result= keras_checker.calculate_invariance(rotated_predictions, normal_predictions)
print(f"Invariance: {result:.4f}")

Invariance: 0.8840


### Resnet - Normal

In [82]:
rotated_loader, normal_loader = keras_checker.transform_test_set(path = "../places/data/test", image_size=(224,224,3), batch_size=128)
rotated_predictions, normal_predictions = keras_checker.predict_models(model = places_ResNet_normal, rotated_testset=rotated_loader, normal_testset=normal_loader)

Found 6250 images belonging to 10 classes.
Found 6250 images belonging to 10 classes.
Rotated Testset:
--------------------------------------------------
49/49 [==============================] - 35s 614ms/step
Accuracy: 0.4046
--------------------------------------------------
--------------------------------------------------
Normal Testset:
--------------------------------------------------
49/49 [==============================] - 25s 521ms/step
Accuracy: 0.8558


In [83]:
result= keras_checker.calculate_invariance(rotated_predictions, normal_predictions)
print(f"Invariance: {result:.4f}")

Invariance: 0.4296
